In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.4` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:0.10.9` 

import $ivy.$                                   // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:0.10.9` 

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)


import org.apache.log4j.{Level, Logger}


In [10]:
import org.apache.spark.sql._
import spark.implicits._

import org.apache.spark.sql._

import spark.implicits._

In [11]:
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .config("spark.testing.memory", 471859200)
    .config("spark.executor.instances", "4")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
}

spark: SparkSession = org.apache.spark.sql.SparkSession@1711aacd

In [12]:
val df = spark.read.format("parquet").load("data/train")

df: DataFrame = [instanceId_userId: int, instanceId_objectType: string ... 167 more fields]

In [12]:
//  посмотрим на список колонок
/*
for (c <- df.columns) {
    println(c)
}
*/

In [13]:
df.createOrReplaceTempView("trainDF")
val df_exp = spark.sql("""
    select t.*, explode(feedback) as fb_exp from trainDF t 
""")
df_exp.createOrReplaceTempView("trainDF_exp")

df_exp: DataFrame = [instanceId_userId: int, instanceId_objectType: string ... 168 more fields]

# Задание 1 - Постройте топ популярных групп на портале

In [14]:
spark.sql("""
    select metadata_ownerId, count(*) as like_cnt from trainDF_exp
    where fb_exp = 'Liked'
    group by metadata_ownerId
    order by like_cnt desc
""").show(10)

+----------------+--------+
|metadata_ownerId|like_cnt|
+----------------+--------+
|           37463|   16726|
|           11222|   12301|
|           18942|   10160|
|           14285|   10024|
|           62833|    9428|
|           26843|    8779|
|           19481|    8717|
|           79619|    8519|
|           11058|    8384|
|           26735|    7770|
+----------------+--------+
only showing top 10 rows



# Задание 2 - Построите гистограммы популярности/активности групп на портале по времени суток

In [15]:
spark.sql("""
    select distinct fb_exp from trainDF_exp
""").show()

+---------+
|   fb_exp|
+---------+
|Commented|
|   Viewed|
| ReShared|
|  Clicked|
|Complaint|
|  Ignored|
| Disliked|
|    Liked|
|  Unliked|
+---------+



In [16]:
val activityDF = spark.sql("""
    select created_hour as created_hour_a, count(*) as cnt_a from (
        select 
            from_unixtime(metadata_createdAt, 'HH') as created_hour
            , fb_exp
        from trainDF_exp 
        where fb_exp <> 'Ignored'
    ) 
    group by created_hour
""").as("df_activity")

val popularityDF = spark.sql("""
    select created_hour as created_hour_p, count(*) as cnt_p from (
        select 
            from_unixtime(metadata_createdAt, 'HH') as created_hour
            , fb_exp
        from trainDF_exp 
        where fb_exp = 'Liked'
    ) 
    group by created_hour
""").as("df_popularity")



activityDF: Dataset[Row] = [created_hour_a: string, cnt_a: bigint]
popularityDF: Dataset[Row] = [created_hour_p: string, cnt_p: bigint]

In [17]:
val joined_df = activityDF.join(
popularityDF
, $"created_hour_a" === $"created_hour_p"
, "inner")

joined_df: DataFrame = [created_hour_a: string, cnt_a: bigint ... 2 more fields]

In [18]:
joined_df.show(24)
joined_df.createOrReplaceTempView("joined_df")

+--------------+------+--------------+------+
|created_hour_a| cnt_a|created_hour_p| cnt_p|
+--------------+------+--------------+------+
|            07|340855|            07|142981|
|            15|328408|            15|137408|
|            11|317125|            11|132435|
|            01|321707|            01|134482|
|            22|307189|            22|128918|
|            16|316001|            16|131774|
|            18|326375|            18|134195|
|            00|312314|            00|131087|
|            17|325420|            17|134038|
|            09|323576|            09|136853|
|            05|320993|            05|135081|
|            19|328067|            19|134972|
|            23|328270|            23|138830|
|            08|312593|            08|131175|
|            03|315376|            03|132778|
|            02|319148|            02|131933|
|            06|313794|            06|131198|
|            20|335332|            20|139733|
|            10|323072|           

In [19]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._



import $ivy.$                                      


import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._



In [20]:
val j_res = spark.sql("""
    select created_hour_a, cnt_a, cnt_p
    from joined_df
    order by created_hour_a desc
""")


var hours = Seq[String]()
var cnt_a = Seq[Int]()
var cnt_p = Seq[Int]()

for (row <- j_res.collect()){
    hours = hours :+ row(0).asInstanceOf[String]
    cnt_a = cnt_a :+ row(1).asInstanceOf[Long].toInt
    cnt_p = cnt_p :+ row(2).asInstanceOf[Long].toInt
}

j_res: DataFrame = [created_hour_a: string, cnt_a: bigint ... 1 more field]
hours: Seq[String] = List(
  "23",
  "22",
  "21",
  "20",
  "19",
  "18",
  "17",
  "16",
  "15",
  "14",
  "13",
  "12",
  "11",
  "10",
  "09",
  "08",
  "07",
  "06",
  "05",
  "04",
  "03",
  "02",
  "01",
  "00"
)
cnt_a: Seq[Int] = List(
  328270,
  307189,
  323341,
  335332,
  328067,
  326375,
  325420,
  316001,
  328408,
  303189,
  322514,
  316304,
  317125,
  323072,
  323576,
  312593,
  340855,
  313794,
  320993,
  325096,
  315376,
  319148,
  321707,
  312314
)
cnt_p: Seq[Int] = List(
  138830,
  128918,
  135906,
  139733,
  134972,
  134195,
  134038,
  131774,
  137408,
  127380,
  135089,
  130546,
  132435,
  135167,
  136853,
  131175,
  142981,
  131198,
  135081,
  136470,
  132778,
  131933,
  134482,
  131087
)

In [21]:
val trace1 = Bar(
  hours,
  cnt_a,
  "Активность",
  text = hours.map(x => "Час " + x),
  // orientation = Orientation.Horizontal,
  marker = Marker(
    color = Color.RGB(49, 130, 189),
    opacity = 0.7))

val trace2 = Bar(
  hours,
  cnt_p,
  "Популярность",
  text = hours.map(x => "Час " + x),
  // orientation = Orientation.Horizontal,
  marker = Marker(
    color = Color.RGB(204, 0, 0),
    opacity = 0.7))

Seq(trace1, trace2).plot("Данные активности и популярности групп по часам")

trace1: Bar = Bar(
  Strings(
    List(
      "23",
      "22",
      "21",
      "20",
      "19",
      "18",
      "17",
      "16",
      "15",
      "14",
      "13",
      "12",
      "11",
      "10",
      "09",
      "08",
      "07",
      "06",
      "05",
      "04",
      "03",
      "02",
      "01",
      "00"
    )
  ),
  Doubles(
    List(
      328270.0,
      307189.0,
      323341.0,
      335332.0,
      328067.0,
      326375.0,
      325420.0,
      316001.0,
...
trace2: Bar = Bar(
  Strings(
    List(
      "23",
      "22",
      "21",
      "20",
      "19",
      "18",
      "17",
      "16",
      "15",
      "14",
      "13",
      "12",
      "11",
      "10",
      "09",
      "08",
      "07",
      "06",
      "05",
      "04",
      "03",
      "02",
      "01",
      "00"
    )
  ),
  Doubles(
    List(
      138830.0,
      128918.0,
      135906.0,
      139733.0,
      134972.0,
      134195.0,
      134038.0,
      131774.0,
...
res20_2: String = 

##### Посольку груфик не отображается в случае, если открыть ноутбук напрямую из гитхаба, продублирую его в виде картинки:

<img src="activity_popularity.png">